# Titanic Survival Prediction - Classification Model

**Student ID:** 202210221  
**Student Name:** Ahmad Abu Ghazaleh  
**Project:** DS&AI Projects - SQA Implementation

## Objective
Predict passenger survival on the Titanic using machine learning classification algorithms.

---

In [3]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
print("✓ Libraries imported successfully")

✓ Libraries imported successfully


## 1. Data Integrity Testing (Unit Testing Phase)
According to SQA Plan: Datasets must be "clean" (0 null values in selected features) before training.

In [4]:
# Load dataset
df = pd.read_csv('../datasets/titanic/train.csv')

print(f"Dataset Shape: {df.shape}")
print(f"\nFirst 5 rows:")
df.head()

Dataset Shape: (891, 12)

First 5 rows:


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## 2. Data Quality Report
Checking for missing values, data types, and basic statistics.

In [5]:
# Check data types and missing values
print("Dataset Information:")
print("="*50)
df.info()

print("\n" + "="*50)
print("Missing Values Count:")
print("="*50)
missing_values = df.isnull().sum()
print(missing_values[missing_values > 0])

Dataset Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB

Missing Values Count:
Age         177
Cabin       687
Embarked      2
dtype: int64


In [6]:
# Statistical summary
print("Statistical Summary:")
df.describe()

Statistical Summary:


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


## 3. Data Cleaning Phase
According to SQA Plan: We must ensure 0 null values in selected features before training.

In [ ]:
# Select features for modeling
# Fixed: Removed 'Cabin' feature due to 77% missing values (CR-001)
selected_features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
target = 'Survived'

# Create working dataframe
df_clean = df[selected_features + [target]].copy()

print(f"Before cleaning: {df_clean.shape[0]} rows")
print(f"Missing values per feature:")
print(df_clean.isnull().sum())

# Handle missing values
# Age: Fill with median
df_clean['Age'].fillna(df_clean['Age'].median(), inplace=True)

# Embarked: Fill with mode (most common value)
df_clean['Embarked'].fillna(df_clean['Embarked'].mode()[0], inplace=True)

# Fare: Fill with median
df_clean['Fare'].fillna(df_clean['Fare'].median(), inplace=True)

# Drop any remaining rows with missing values
df_clean = df_clean.dropna()

print(f"\n✓ Data cleaned successfully!")
print(f"After cleaning: {df_clean.shape[0]} rows")
print(f"Data retention: {df_clean.shape[0]/df.shape[0]*100:.1f}%")
print(f"Missing values: {df_clean.isnull().sum().sum()}")

✓ Data cleaned successfully!
Final dataset shape: (183, 9)
Missing values: 0


In [8]:
# Encode categorical variables
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df_clean['Sex'] = le.fit_transform(df_clean['Sex'])
df_clean['Embarked'] = le.fit_transform(df_clean['Embarked'])
df_clean['Cabin'] = le.fit_transform(df_clean['Cabin'])

print("✓ Features encoded successfully")
df_clean.head()

✓ Features encoded successfully


,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Survived
1,1,0,38.0,1,0,71.2833,72,0,1
3,1,0,35.0,1,0,53.1000,48,2,1
6,1,1,54.0,0,0,51.8625,117,2,0
10,3,0,4.0,1,1,16.7000,131,2,1
11,1,0,58.0,0,0,26.5500,43,2,1
